In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import joblib

In [42]:
df = pd.read_csv("s3://german-credit-20221001/datos/original/german_credit_data.csv")

In [43]:
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [45]:
df.describe()

,Unnamed: 0,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,35.546000,1.904000,3271.258000,20.903000
std,288.819436,11.375469,0.653614,2822.736876,12.058814
min,0.000000,19.000000,0.000000,250.000000,4.000000
25%,249.750000,27.000000,2.000000,1365.500000,12.000000
50%,499.500000,33.000000,2.000000,2319.500000,18.000000
75%,749.250000,42.000000,2.000000,3972.250000,24.000000
max,999.000000,75.000000,3.000000,18424.000000,72.000000


In [46]:
df["Risk"].value_counts()

good    700
bad     300
Name: Risk, dtype: int64

In [47]:
train, test = train_test_split(df, test_size=0.3, random_state=42, stratify=df["Risk"])

In [48]:
train["Risk"].value_counts()/train["Risk"].count()

good    0.7
bad     0.3
Name: Risk, dtype: float64

In [49]:
test["Risk"].value_counts()/test["Risk"].count()

good    0.7
bad     0.3
Name: Risk, dtype: float64

In [50]:
train.to_csv("s3://german-credit-20221001/datos/train/train.csv", index=False)

In [51]:
test.to_csv("s3://german-credit-20221001/datos/test/test.csv")

In [52]:
pd.crosstab(index=train["Housing"], columns=train["Risk"], margins=True)

Risk,bad,good,All
Housing,,,
free,29,45,74
own,133,371,504
rent,48,74,122
All,210,490,700


In [53]:
pd.crosstab(index=train["Duration"], columns=train["Risk"], margins=True)

Risk,bad,good,All
Duration,,,
4,0,6,6
6,7,49,56
7,0,2,2
8,1,3,4
9,7,26,33
10,3,19,22
11,0,5,5
12,37,83,120
13,0,3,3


In [54]:
tabla_sex =pd.crosstab(index=train["Sex"], columns=train["Risk"], margins=True)

In [55]:
tabla_sex["proba"] = tabla_sex.iloc[:,1]/tabla_sex.iloc[:,2]

In [56]:
tabla_sex

Risk,bad,good,All,proba
Sex,,,,
female,76,138,214,0.64486
male,134,352,486,0.72428
All,210,490,700,0.70000


In [57]:
##Preprosesamiento
from sklearn.impute import SimpleImputer

In [58]:
#Se imputan variables que no contienen datos, esto es para producción
#Age","Credit amount","Duration",
X_train = train[["Saving accounts","Checking account"]]

In [59]:
imputer = SimpleImputer(strategy="constant", fill_value='UPS')

In [60]:
imputer.fit(X_train)

SimpleImputer(fill_value='UPS', strategy='constant')

In [61]:
imputer.statistics_

array(['UPS', 'UPS'], dtype=object)

In [62]:
imputer.transform([['moderate',np.nan],[np.nan,np.nan]])

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([['moderate', 'UPS'],
       ['UPS', 'UPS']], dtype=object)

In [63]:
imputer.fit_transform(X_train)

array([['little', 'rich'],
       ['little', 'UPS'],
       ['UPS', 'moderate'],
       ...,
       ['little', 'UPS'],
       ['little', 'UPS'],
       ['little', 'little']], dtype=object)

## Prueba con KNN

In [64]:
df2 = pd.DataFrame( [[40,19,3.0],[42,20,3.1],[44,21,np.nan],[45,23,4.1],[39,25,5.0],[80,27,np.nan],[82,30,4.8]], columns=["Peso","Edad","Nota"])

In [65]:
from sklearn.impute import KNNImputer

In [66]:
imputer = KNNImputer(n_neighbors=4)

In [67]:
imputer.fit(df2)

KNNImputer(n_neighbors=4)

In [68]:
imputer.transform(df2)

array([[40.  , 19.  ,  3.  ],
       [42.  , 20.  ,  3.1 ],
       [44.  , 21.  ,  3.8 ],
       [45.  , 23.  ,  4.1 ],
       [39.  , 25.  ,  5.  ],
       [80.  , 27.  ,  3.75],
       [82.  , 30.  ,  4.8 ]])

## Escalado

In [69]:
 from sklearn.preprocessing import StandardScaler, RobustScaler

In [70]:
X_train = train[["Age","Credit amount","Duration"]]

In [71]:
scaler = StandardScaler()

In [72]:
scaler.fit(X_train[["Age"]])

StandardScaler()

In [73]:
scaler.mean_

array([35.40285714])

In [74]:
scaler.var_ ** 0.5

array([11.23479253])

In [75]:
scaler.inverse_transform([[-0.39189483]])

array([[31.00000003]])

In [76]:
X_train["Age"].quantile(0.5)

33.0

In [77]:
X_train["Age"] - X_train["Age"].mean() / (X_train["Age"].quantile(0.75) - X_train["Age"].quantile(0.25)*1.5)

328   -16.20381
891    -1.20381
255   -20.20381
243   -20.20381
492   -20.20381
         ...   
73     -6.20381
401   -19.20381
769    15.79619
2       1.79619
617   -10.20381
Name: Age, Length: 700, dtype: float64

In [78]:
X_train["Age"].median()

33.0

In [87]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [88]:
encoder = OneHotEncoder(sparse=False, drop ='first')
encoder.fit_transform(train[["Sex"]])

array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],

In [89]:
encoder.categories_

[array(['female', 'male'], dtype=object)]

In [90]:
encoder.inverse_transform([[0]])

array([['female']], dtype=object)

In [92]:
from sklearn.pipeline import Pipeline

In [93]:
numeric_pipeline = Pipeline(
    [
        ('Imputación con la media', SimpleImputer(strategy='mean')),
        ('Escalado minmax', MinMaxScaler())
    ]
)

In [94]:
numeric_pipeline.fit(train[["Age","Credit amount","Duration"]])

Pipeline(steps=[('Imputación con la media', SimpleImputer()),
                ('Escalado minmax', MinMaxScaler())])

In [95]:
numeric_pipeline[0].statistics_

array([  35.40285714, 3236.12142857,   21.04857143])

In [96]:
numeric_pipeline[1].min_

array([-0.36363636, -0.01375592, -0.05882353])

In [97]:
numeric_pipeline[1].data_max_

array([   75., 18424.,    72.])

In [98]:
train[["Age","Credit amount","Duration"]].max()

Age                 75
Credit amount    18424
Duration            72
dtype: int64

In [99]:
numeric_pipeline.transform(train[["Age","Credit amount","Duration"]])

array([[0.2       , 0.23236492, 0.47058824],
       [0.47272727, 0.08688236, 0.16176471],
       [0.12727273, 0.3944096 , 0.82352941],
       ...,
       [0.78181818, 0.07730824, 0.11764706],
       [0.52727273, 0.10157368, 0.11764706],
       [0.30909091, 0.18851106, 0.02941176]])

In [100]:
numeric_pipeline.transform([[np.nan, np.nan, np.nan]])

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[0.28005195, 0.16430733, 0.25071429]])

In [101]:
#Sex, Purpose, Housing
categorical_pipeline = Pipeline(
    [
        ('Imputación con la moda', SimpleImputer(strategy='most_frequent')),
        ('Escalado OneHotEncoder', OneHotEncoder(sparse=False))
    ]
)

In [102]:
categorical_pipeline.fit(train[["Sex","Purpose","Housing"]])

Pipeline(steps=[('Imputación con la moda',
                 SimpleImputer(strategy='most_frequent')),
                ('Escalado OneHotEncoder', OneHotEncoder(sparse=False))])

In [103]:
categorical_pipeline.transform(train[["Sex","Purpose","Housing"]])

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [104]:
categorical_pipeline[1].categories_

[array(['female', 'male'], dtype=object),
 array(['business', 'car', 'domestic appliances', 'education',
        'furniture/equipment', 'radio/TV', 'repairs', 'vacation/others'],
       dtype=object),
 array(['free', 'own', 'rent'], dtype=object)]

In [105]:
from sklearn.compose import ColumnTransformer

In [106]:
column_transformer = ColumnTransformer(
    [
        ("numeric pipeline", numeric_pipeline, [0,1,2]),
        ("categorical pipeline", categorical_pipeline, [3,4,5])
    ]
)

In [107]:
column_transformer.fit(train[["Age","Credit amount","Duration", "Sex","Purpose","Housing"]])

ColumnTransformer(transformers=[('numeric pipeline',
                                 Pipeline(steps=[('Imputación con la media',
                                                  SimpleImputer()),
                                                 ('Escalado minmax',
                                                  MinMaxScaler())]),
                                 [0, 1, 2]),
                                ('categorical pipeline',
                                 Pipeline(steps=[('Imputación con la moda',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Escalado OneHotEncoder',
                                                  OneHotEncoder(sparse=False))]),
                                 [3, 4, 5])])

In [108]:
column_transformer.transform(train[["Age","Credit amount","Duration", "Sex","Purpose","Housing"]])

array([[0.2       , 0.23236492, 0.47058824, ..., 0.        , 1.        ,
        0.        ],
       [0.47272727, 0.08688236, 0.16176471, ..., 0.        , 1.        ,
        0.        ],
       [0.12727273, 0.3944096 , 0.82352941, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.78181818, 0.07730824, 0.11764706, ..., 0.        , 1.        ,
        0.        ],
       [0.52727273, 0.10157368, 0.11764706, ..., 0.        , 1.        ,
        0.        ],
       [0.30909091, 0.18851106, 0.02941176, ..., 0.        , 0.        ,
        1.        ]])

In [109]:
joblib.dump(column_transformer,'processing.joblib')

['processing.joblib']

## Evaluación de modelos con validación cruzada

In [81]:
##Versión 1, evaluar el modelo

In [110]:
X_train = train[["Age","Credit amount","Duration", "Sex","Purpose","Housing"]]

In [111]:
y_train = train["Risk"]

In [112]:
X_train_processed = column_transformer.transform(X_train)

In [114]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

In [116]:
y_train_transform = (y_train=='good').astype("int")

In [118]:
results = cross_validate(DecisionTreeClassifier(),
                        X_train_processed, y_train_transform, cv=10,
                        scoring=["accuracy","f1"])

In [120]:
#En el artículo se debe colocar el resultado mostrando la media y la desviación standard
print(results["test_accuracy"].mean(), "+-", results["test_accuracy"].std())

0.6228571428571429 +- 0.05118992247762548


## Versión 2 evaluar el preprocesamiento incluyéndolo en el pipeline (mejor que la versión 1)

In [121]:
final_pipeline = Pipeline(
    [
        ('Pipeline de preprocesamiento', column_transformer),
        ('Modelo de predicción de Árbol', DecisionTreeClassifier())
    ]
)

In [127]:
results = cross_validate(final_pipeline,
                        X_train, y_train_transform, cv=10,
                        scoring=["accuracy","f1"])

In [129]:
#En el artículo se debe colocar el resultado mostrando la media y la desviación standard
print(results["test_accuracy"].mean(), "+-", results["test_accuracy"].std())

0.6214285714285713 +- 0.03951233338839809


## Versión 3 evaluar el preprocesamiento incluyéndolo en el pipeline (mejor que la versión 2)

In [145]:
final_pipeline = Pipeline(
    [
        ('Pipeline de preprocesamiento', column_transformer),
        ('Modelo de predicción de Árbol', DecisionTreeClassifier(max_depth=3))
    ]
)

In [146]:
results = cross_validate(final_pipeline,
                        X_train, y_train_transform, cv=10,
                        scoring=["accuracy","f1"])

In [148]:
#En el artículo se debe colocar el resultado mostrando la media y la desviación standard
print(results["test_accuracy"].mean(), "+-", results["test_accuracy"].std())

0.7228571428571429 +- 0.014568627181693663


## Versión 4 con ajuste de hiperparámetros

In [152]:
from sklearn.model_selection import GridSearchCV

In [159]:
grid = {'max_depth':[2,3,5,10]}

In [160]:
grid_search = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid=grid,
    cv=10,
    scoring="accuracy"
)

In [161]:
grid_search.fit(X_train_processed, y_train_transform)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 5, 10]}, scoring='accuracy')

In [164]:
grid_search.best_score_

0.7214285714285713

In [165]:
grid_search.best_params_

{'max_depth': 3}